In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MultiLabelBinarizer
import mlflow
import mlflow.keras 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import numpy as np
import tensorflow_hub as hub
import mlflow.pyfunc
import os
import tensorflow as tf
import pickle
from sklearn.model_selection import train_test_split


In [ ]:

mlflow.set_experiment("Text_Processing_Experiment")




In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('db/questions_db.csv')
print(df.head())
# pour réduire l'échantillon si nécessaire
df = df.sample(frac=0.1, random_state=42) 

del df['FavoriteCount'] # efface une colonne en attendant



In [ ]:
mlflow.start_run(run_name="Full Pipeline")

# Logue les paramètres globaux
mlflow.log_param("data_sample_fraction", 0.1)
mlflow.log_param("num_samples", len(df))
mlflow.log_param("num_features", df.shape[1])


In [ ]:

# transforme les tags sous forme de chaîne de caractères en liste
def process_tags(tags):
    if isinstance(tags, str):
        
        # Enlève les symboles '<>' entourant les tags et les divise en une liste
        return tags.strip('<>').split('><')
    else:
        return []
    

# Applique le traitement des tags à la colonne 'Tags' du DataFrame
df['Tags'] = df['Tags'].apply(process_tags)

In [ ]:

# Vérifie les valeurs manquantes
def check_missing_values(df):
    missing_values = df.isnull().sum()
    print("Nombre de valeurs manquantes par colonne:")
    print(missing_values)
    if missing_values.any():
        print("\nColonnes avec valeurs manquantes:")
        print(missing_values[missing_values > 0])
    else:
        print("Aucune valeur manquante trouvée.")


check_missing_values(df)

In [ ]:
# Une liste des tags présents dans le DataFrame
all_tags = [tag for tags in df['Tags'] for tag in tags]

if all_tags:
    # Si oui, affiche le nombre total de tags extraits
    print(f"Nombre total de tags extraits: {len(all_tags)}")
else:
    print("Erreur : Aucun tag n'a été extrait")

# Compte la fréquence de chaque tag et sélectionne les 50 tags les plus fréquents
tag_counts = pd.Series(all_tags).value_counts().head(50)

# Les convertie en une liste
top_tags = list(tag_counts.index)

# Filtre les questions pour ne conserver que celles avec les top 50 tags
df['Filtered_Tags'] = df['Tags'].apply(lambda tags: [tag for tag in tags if tag in top_tags])

# Supprime les questions sans tags dans le top 50
df = df[df['Filtered_Tags'].apply(lambda x: len(x) > 0)]

print(top_tags)
plt.figure(figsize=(12, 18))
sns.barplot(x=tag_counts.values, y=tag_counts.index)
plt.title('Top 50 des Tags les plus fréquents')
plt.xlabel('Fréquence')
plt.ylabel('Tags')
plt.show()


In [ ]:
import re

# Supprime balises HTML, caractères spéciaux, emojis, chiffres, mots de moins de 2 caractères
def clean_text(text):
    if isinstance(text, str):
        # Supprime les caractères spéciaux (non alphanumériques et non espaces)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Supprime les emojis
        text = re.sub(r'[^\w\s,]', '', text)
        # Supprime les chiffres
        text = re.sub(r'\d+', '', text)
        # Supprime les mots à - de 2 caractères
        text = ' '.join([word for word in text.split() if len(word) > 2])
        # Supprime les balises HTML (y compris les balises mal formatées)
        text = re.sub(r'<[^>]+>', '', text)
    return text

# Applique la colonne 'Title'
df['Cleaned_Title'] = df['Title'].apply(clean_text)

# Applique la fonction à la colonne 'Body'
df['Cleaned_Body'] = df['Body'].apply(clean_text)

# Vérifie le résultat
print(df[['Title', 'Cleaned_Title', 'Body', 'Cleaned_Body']].head())

# Vérifie les tags
print("Vérification des tags")
print(df['Tags'].head(10))

# Contenu nettoyé
print('cleaned body', df['Cleaned_Body'].values)

df.head(10)


In [ ]:
def add_features(df):
    # Longueur du titre
    df['Title_length'] = df['Title'].apply(
        lambda x: len(x.split()) if isinstance(x, str) else 0)
    # Nombre de mots dans le corps
    df['Body_word_count'] = df['Body'].apply(
        lambda x: len(x.split()) if isinstance(x, str) else 0)
    # Nombre de tags
    df['Tag_count'] = df['Tags'].apply(
        lambda x: len(x) if isinstance(x, list) else 0)
    return df


# Ajoute les features supplémentaires
df = add_features(df)
print(df.head(10))

In [ ]:
df.info()

## Analyse multivariée


In [ ]:
cols_to_analyze = ['Score', 'ViewCount', 'AnswerCount',
                   'Title_length', 'Body_word_count', 'Tag_count']
score_col = 'Score'
view_col = 'ViewCount'
favorite_col = 'FavoriteCount'

In [ ]:
def analyse_univariee_multivariee(df, cols):
    print("Analyse Univariée")
    for column in cols:
        if column in df.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(df[column], kde=True)
            plt.title(f'Distribution de {column}')
            plt.xlabel(column)
            plt.ylabel('Fréquence')
            plt.show()

    print("Analyse Multivariée")
    sns.pairplot(df[cols].select_dtypes(include=['int64', 'float64']))
    plt.show()


analyse_univariee_multivariee(df, cols_to_analyze)

In [ ]:
print(df.describe())

In [ ]:
def correlation_variables(df, cols):
    corr_matrix = df[cols].corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title('Matrice de Corrélation')
    return corr_matrix

# Utilisation avec suivi MLflow
with mlflow.start_run(run_name="Correlation Analysis", nested=True):
    # Paramètres logués
    mlflow.log_param("analyzed_columns", cols_to_analyze)

    # Calcul et visualisation de la matrice de corrélation
    corr_matrix = correlation_variables(df, cols_to_analyze)

    # Sauvegarde et log du graphique de la matrice de corrélation
    corr_matrix_plot_path = "correlation_matrix.png"
    plt.savefig(corr_matrix_plot_path)
    mlflow.log_artifact(corr_matrix_plot_path)

    # Calcul et log de la métrique de corrélation moyenne
    mean_corr = corr_matrix.mean().mean()
    mlflow.log_metric("mean_correlation", mean_corr)

mlflow.end_run()


### Tokenization + stopwords + lemmatization

In [ ]:
def treatments_text(text, remove_stopwords=True, lemmatize=True):
    # Tokenization
    tokens = word_tokenize(text)

    if remove_stopwords:
        # Suppression des stop words
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word.lower() not in stop_words]

    if lemmatize:
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

# Nettoyage du DataFrame
def clean_dataframe(df, remove_stopwords=True, lemmatize=True):
    df['Cleaned_Title'] = df['Title'].apply(lambda x: treatments_text(x, remove_stopwords, lemmatize))
    df['Cleaned_Body'] = df['Body'].apply(lambda x: treatments_text(x, remove_stopwords, lemmatize))
    return df

# Suivi MLflow
with mlflow.start_run(run_name="Text Cleaning", nested=True):
    # Paramètres de nettoyage
    remove_stopwords = True
    lemmatize = True
    
    # Log des paramètres
    mlflow.log_param("remove_stopwords", remove_stopwords)
    mlflow.log_param("lemmatize", lemmatize)

    # Nettoyage du DataFrame
    df = clean_dataframe(df, remove_stopwords=remove_stopwords, lemmatize=lemmatize)

    # Log d'un échantillon des données nettoyées
    sample_cleaned = df[['Title', 'Cleaned_Title', 'Body', 'Cleaned_Body']].head(10)
    sample_cleaned_path = "cleaned_data_sample.csv"
    sample_cleaned.to_csv(sample_cleaned_path, index=False)
    mlflow.log_artifact(sample_cleaned_path)

print(df[['Title', 'Cleaned_Title', 'Body', 'Cleaned_Body']].head())
print('clean', df[['Cleaned_Body']].values)
mlflow.end_run()


### Différentes méthodes pour traiter les mots

In [ ]:
def transform_text_to_bow(df, max_df=0.95, min_df=2, n_components=100):
    if 'Cleaned_Body' not in df.columns:
        raise ValueError("La colonne 'Cleaned_Body' est manquante dans le DataFrame")
    
    # Bag of Words : compte les mots et les représente en format numérique reflétant leur nombre
    vectorizer = CountVectorizer(max_df=max_df, min_df=min_df)
    X = vectorizer.fit_transform(df['Cleaned_Body'])

    # Réduction de dimension avec TruncatedSVD : réduit la dimensionnalité des vecteurs BoW
    svd = TruncatedSVD(n_components=n_components)
    X_reduced = svd.fit_transform(X)

    # Transformation des tags : convertit les tags en une forme utilisable pour l'apprentissage supervisé
    mlb = MultiLabelBinarizer(classes=top_tags)
    y = mlb.fit_transform(df['Filtered_Tags'])

    return X_reduced, y, vectorizer, svd, mlb

# Utilisation de la fonction avec suivi MLflow
with mlflow.start_run(run_name="BoW Transformation", nested=True):
    # Paramètres de transformation
    max_df = 0.95
    min_df = 2
    n_components = 100
    
    # Log des paramètres
    mlflow.log_param("max_df", max_df)
    mlflow.log_param("min_df", min_df)
    mlflow.log_param("n_components", n_components)

    # Transformation du texte
    X_reduced, y, vectorizer, svd, mlb = transform_text_to_bow(df, max_df=max_df, min_df=min_df, n_components=n_components)
    
    # Log des métriques
    explained_variance = svd.explained_variance_ratio_.sum()
    mlflow.log_metric("explained_variance", explained_variance)
    mlflow.log_metric("X_reduced_num_rows", X_reduced.shape[0])  # Nombre de lignes
    mlflow.log_metric("X_reduced_num_columns", X_reduced.shape[1])  # Nombre de colonnes
    mlflow.log_metric("y_num_rows", y.shape[0])  # Nombre de lignes
    mlflow.log_metric("y_num_columns", y.shape[1])  # Nombre de colonnes

    # Enregistrement des objets comme artefacts
    vectorizer_path = "model/vectorizer.pkl"
    svd_path = "model/svd.pkl"
    mlb_path = "model/mlb.pkl"
    
    with open(vectorizer_path, 'wb') as f:
        pickle.dump(vectorizer, f)
    with open(svd_path, 'wb') as f:
        pickle.dump(svd, f)
    with open(mlb_path, 'wb') as f:
        pickle.dump(mlb, f)
    
    mlflow.log_artifact(vectorizer_path)
    mlflow.log_artifact(svd_path)
    mlflow.log_artifact(mlb_path)

mlflow.end_run()


In [ ]:
def train_word2vec(df, column='Cleaned_Body', vector_size=50, window=5, min_count=3, workers=4):
     # Tokenize les textes dans la colonne spécifiée (par défaut 'Cleaned_Body')
    sentences = df[column].apply(word_tokenize).tolist() # Chaque texte est transformé en une liste de mots


    # Entraîne un modèle Word2Vec sur les phrases tokenisées
    # vector_size: la dimension des vecteurs d'embedding
    # window: la taille de la fenêtre contextuelle utilisée pour les mots voisins
    # min_count: le nombre minimum d'occurrences d'un mot pour être inclus dans le modèle
    # workers: le nombre de threads utilisés pour l'entraînement
    word2vec_model = Word2Vec(sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    
    def document_vector(doc):
        doc = [word for word in doc if word in word2vec_model.wv.index_to_key]
        return np.mean(word2vec_model.wv[doc], axis=0) if doc else np.zeros(vector_size)
    
    df['word2vec_vector'] = df[column].apply(lambda x: document_vector(word_tokenize(x)))
    
    X_word2vec = np.vstack(df['word2vec_vector'].values)
    model_path = 'model/word2vec'
    word2vec_model.save(model_path)
    return X_word2vec, model_path

with mlflow.start_run(run_name="Word2Vec Training", nested=True):
    # Paramètres pour l'entraînement Word2Vec
    vector_size = 50
    window = 5
    min_count = 3
    workers = 4
    
    # Log des paramètres
    mlflow.log_param("vector_size", vector_size)
    mlflow.log_param("window", window)
    mlflow.log_param("min_count", min_count)
    mlflow.log_param("workers", workers)

    # Entraînement Word2Vec
    X_word2vec, model_path = train_word2vec(df, column='Cleaned_Body', vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    
    # Log de la forme des vecteurs
    mlflow.log_metric("word2vec_num_rows", X_word2vec.shape[0])  # Nombre de lignes
    mlflow.log_metric("word2vec_num_columns", X_word2vec.shape[1])  # Nombre de colonnes

    # Calcul et log de la moyenne des longueurs des vecteurs
    vector_lengths = np.linalg.norm(X_word2vec, axis=1)
    mean_vector_length = np.mean(vector_lengths)
    mlflow.log_metric("mean_vector_length", mean_vector_length)
    
    # Sauvegarde du modèle Word2Vec avec MLflow
    mlflow.log_artifact(model_path, artifact_path="models/word2vec")

mlflow.end_run()


### Différentes méthodes pour traiter les mots

In [ ]:
# Charger les données nettoyées (Cleaned_Body)
df_cleaned_bodies = df['Cleaned_Body'].tolist()

# Charge le modèle USE depuis TensorFlow Hub
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_layer = hub.KerasLayer(model_url, trainable=False)

# Applique le modèle USE pour obtenir les embeddings
X_use = use_layer(df_cleaned_bodies)

# Converti les tensors en NumPy
X_use_np = np.array(X_use)

# Split des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_use_np, y, test_size=0.2, random_state=42)

# Définir le modèle USE
model_use = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_use_np.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(y.shape[1], activation='sigmoid')
])

# Compiler le modèle
model_use.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Définir un callback pour éviter le surajustement
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Entraîner le modèle et sauvegarder l'historique
history_use = model_use.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            epochs=10,
                            batch_size=32,
                            callbacks=[early_stopping])


In [ ]:
X_reduced, y, vectorizer, svd, mlb = transform_text_to_bow(df)

# Vérification que X_reduced et y ne sont pas None
print("Shape of X_reduced:", X_reduced.shape if X_reduced is not None else "X_reduced is None")
print("Shape of y:", y.shape if y is not None else "y is None")


In [ ]:
import pickle
import tensorflow as tf


with open('model/df.pkl', 'wb') as f:
    pickle.dump(df, f)


# Sauvegarde des objets pour BoW + SVD
with open('model/X_reduced.pkl', 'wb') as f:
    pickle.dump(X_reduced, f)

with open('model/y.pkl', 'wb') as f:
    pickle.dump(y, f)

with open('model/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('model/svd.pkl', 'wb') as f:
    pickle.dump(svd, f)

with open('model/top_tags.pkl', 'wb') as f:
    pickle.dump(top_tags, f)

# Sauvegarde des objets pour Word2Vec
X_word2vec, _ = train_word2vec(df, column='Cleaned_Body')
with open('model/X_word2vec.pkl', 'wb') as f:
    pickle.dump(X_word2vec, f)

# Sauvegarde des embeddings USE dans un fichier pickle
with open('model/X_use_embeddings.pkl', 'wb') as f:
    pickle.dump(X_use_np, f)

print("Embeddings USE sauvegardés avec succès dans 'model/X_use_embeddings.pkl'")


with open('model/X_use_embeddings.pkl', 'wb') as f:
    pickle.dump(X_use_np, f)

with open('model/y.pkl', 'wb') as f:
    pickle.dump(y, f)

with open('model/history_use.pkl', 'wb') as f:
    pickle.dump(history_use.history, f)


# Sauvegarde du modèle USE en format SavedModel
saved_model_path = "saved_models/use_model"
model_use.save('saved_models/use_model.keras')

print("Entraînement terminé et objets sauvegardés.")

